In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from pathlib import Path

class MarvelActorRecognizer:
    def __init__(self, img_height=224, img_width=224):
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        self.class_names = []
        
    def create_model(self, num_classes):
        model = models.Sequential([
            layers.Input(shape=(self.img_height, self.img_width, 3)),  # Explicit input layer
            layers.Conv2D(32, (3,3), activation='relu'),
            layers.MaxPooling2D(2,2),
            layers.Conv2D(64, (3,3), activation='relu'),
            layers.MaxPooling2D(2,2),
            layers.Conv2D(128, (3,3), activation='relu'),
            layers.MaxPooling2D(2,2),
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dense(num_classes, activation='softmax')
        ])
        return model
    
    def prepare_data(self, data_dir):
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )

        train_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=32,
            class_mode='categorical',
            subset='training'
        )
        
        val_generator = train_datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=32,
            class_mode='categorical',
            subset='validation'
        )
        
        self.class_names = list(train_generator.class_indices.keys())
        return train_generator, val_generator
    
    def train(self, train_gen, val_gen, epochs=20):
        self.model = self.create_model(len(self.class_names))
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        history = self.model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(patience=5),
                tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3),
                tf.keras.callbacks.ModelCheckpoint('../models/marvel_model.h5', save_best_only=True)
            ]
        )
        return history
    
    @staticmethod
    def get_character(actor):
        character_map = {
            'robert_downey_jr': 'Tony Stark/Iron Man',
            'chris_evans': 'Steve Rogers/Captain America',
            'chris_hemsworth': 'Thor',
            'scarlett_johansson': 'Natasha Romanoff/Black Widow',
            'mark_ruffalo': 'Bruce Banner/Hulk',
            'jeremy_renner': 'Clint Barton/Hawkeye'
        }
        return character_map.get(actor, 'Unknown Character')

# Training Execution
if __name__ == '__main__':
    recognizer = MarvelActorRecognizer()
    train_gen, val_gen = recognizer.prepare_data('../dataset')
    history = recognizer.train(train_gen, val_gen)

Found 469 images belonging to 6 classes.
Found 117 images belonging to 6 classes.


d:\marvel-character-recognition\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.1866 - loss: 5.1306

15/15 ━━━━━━━━━━━━━━━━━━━━ 150s 9s/step - accuracy: 0.1866 - loss: 5.0237 - val_accuracy: 0.2137 - val_loss: 1.7799 - learning_rate: 0.0010
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2200 - loss: 1.7762

15/15 ━━━━━━━━━━━━━━━━━━━━ 134s 9s/step - accuracy: 0.2195 - loss: 1.7758 - val_accuracy: 0.2222 - val_loss: 1.7578 - learning_rate: 0.0010
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2256 - loss: 1.7567

15/15 ━━━━━━━━━━━━━━━━━━━━ 117s 8s/step - accuracy: 0.2276 - loss: 1.7558 - val_accuracy: 0.2906 - val_loss: 1.6911 - learning_rate: 0.0010
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2963 - loss: 1.6530

15/15 ━━━━━━━━━━━━━━━━━━━━ 135s 7s/step - accuracy: 0.2964 - loss: 1.6540 - val_accuracy: 0.3248 - val_loss: 1.6724 - learning_rate: 0.0010
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 96s 6s/step - accuracy: 0.3261 - loss: 1.6838 - val_accuracy: 0.3077 - val_loss: 1.7273 - learning_rate: 0.0010
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 94s 6s/step - accuracy: 0.3482 - loss: 1.6028 - val_accuracy: 0.2479 - val_loss: 1.8033 - learning_rate: 0.0010
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3480 - loss: 1.6677

15/15 ━━━━━━━━━━━━━━━━━━━━ 104s 7s/step - accuracy: 0.3479 - loss: 1.6655 - val_accuracy: 0.3846 - val_loss: 1.5929 - learning_rate: 0.0010
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 95s 6s/step - accuracy: 0.3799 - loss: 1.5423 - val_accuracy: 0.3504 - val_loss: 1.6373 - learning_rate: 0.0010
Epoch 9/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3742 - loss: 1.5732

15/15 ━━━━━━━━━━━━━━━━━━━━ 105s 7s/step - accuracy: 0.3747 - loss: 1.5720 - val_accuracy: 0.4188 - val_loss: 1.5237 - learning_rate: 0.0010
Epoch 10/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 94s 6s/step - accuracy: 0.4274 - loss: 1.4562 - val_accuracy: 0.3248 - val_loss: 1.6124 - learning_rate: 0.0010
Epoch 11/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 90s 6s/step - accuracy: 0.3571 - loss: 1.5236 - val_accuracy: 0.4103 - val_loss: 1.6033 - learning_rate: 0.0010
Epoch 12/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.3751 - loss: 1.5096 - val_accuracy: 0.4103 - val_loss: 1.5350 - learning_rate: 0.0010
Epoch 13/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.4558 - loss: 1.3825 - val_accuracy: 0.3932 - val_loss: 1.6126 - learning_rate: 2.0000e-04
Epoch 14/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5230 - loss: 1.2945

15/15 ━━━━━━━━━━━━━━━━━━━━ 98s 7s/step - accuracy: 0.5217 - loss: 1.2954 - val_accuracy: 0.4615 - val_loss: 1.4467 - learning_rate: 2.0000e-04
Epoch 15/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 880s 62s/step - accuracy: 0.4743 - loss: 1.3236 - val_accuracy: 0.4274 - val_loss: 1.5268 - learning_rate: 2.0000e-04
Epoch 16/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 97s 6s/step - accuracy: 0.4614 - loss: 1.2912 - val_accuracy: 0.4530 - val_loss: 1.5084 - learning_rate: 2.0000e-04
Epoch 17/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 96s 6s/step - accuracy: 0.4499 - loss: 1.4387 - val_accuracy: 0.4274 - val_loss: 1.4914 - learning_rate: 2.0000e-04
Epoch 18/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 95s 6s/step - accuracy: 0.4510 - loss: 1.3928 - val_accuracy: 0.4359 - val_loss: 1.4550 - learning_rate: 4.0000e-05
Epoch 19/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 96s 6s/step - accuracy: 0.5475 - loss: 1.2602 - val_accuracy: 0.4017 - val_loss: 1.4837 - learning_rate: 4.0000e-05
